In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn import feature_extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
stop_words = feature_extraction.text.ENGLISH_STOP_WORDS

In [2]:
df = pd.read_csv("mbti_1.csv")
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [10]:
def preprocess(text):
    text = text.split("|||")
    temp = ''
    for i in text:
        if not re.match(".*http.*", i):
            temp += i
    text = temp
    text = re.sub(r'[^\w\s]', '', text) #remove punctuations
    text = re.sub(r'\d+', '', text) #remove numbers
    text = text.lower() #lowercase
    text = " ".join(text.split()) #stripWhitespace
    text = text.split()
    text = [x for x in text if len(x)>2]
    text = [x for x in text if x not in stop_words] #remove stopwords
    text = [x for x in text if x not in [""]] #remove task specific stopwords
    text = " ".join(text)
    return(text)

In [11]:
df["preprocess"] = df["posts"].apply(lambda x: preprocess(x))

In [7]:
df.to_csv("processed.csv", index = False)

In [14]:
vectorizer = TfidfVectorizer(binary=True, min_df=50)
x = vectorizer.fit_transform(df['preprocess'].values.astype('U'))
df_bow = pd.DataFrame(x.toarray(),columns=vectorizer.get_feature_names())

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [19]:
df_bow["y"] = df["type"]

In [21]:
df_bow.to_csv("bow.csv", index = False)